In [1]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Input
from keras import applications
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, EarlyStopping
import pandas as pd
import yaml
from tqdm import tqdm
import cv2

Using TensorFlow backend.


In [2]:
img_width, img_height = 150, 150
image_size = (img_width, img_height)

# Load image data and labels
- create test set and validation set

In [3]:
df_train = pd.read_csv('../data/train_v2.csv')

with open('../label_maps.yml', 'r') as lablels:
    label_data = yaml.load(lablels)
inv_label_map = label_data['inv_label_map']
label_map = label_data['label_map']

x_train = []
y_train = []

for f, tags in tqdm(df_train.values, miniters=1000):
    img = cv2.imread('../data/train-jpg/{}.jpg'.format(f))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    x_train.append(cv2.resize(img, image_size))
    y_train.append(targets)
    
    
y_train = np.array(y_train, np.uint8)
x_train = np.array(x_train, np.uint8)

100%|██████████| 40479/40479 [02:00<00:00, 334.94it/s]


In [5]:
import numpy as np
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.15, random_state=42)
nb_train_samples = len(x_train)
nb_validation_samples = len(x_valid)
print(x_train.shape)
print(y_train.shape)

(29245, 150, 150, 3)
(29245, 17)


In [5]:
#x_train = x_train[:10]
#x_valid = x_valid[:5]
#y_train = y_train[:10]
#y_valid = y_valid[:5]
#nb_train_samples = len(x_train)
#nb_validation_samples = len(x_valid)
#epochs = 10
#batch_size = 256

# Train the top layer
- Creates feature vector
- trains top layer

In [6]:
def save_bottlebeck_features():
    batch_size = 256
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    print('Loading VGG16 model')
    model = applications.VGG16(include_top=False, weights='imagenet')

    # generator does not shuffle images, the output should be in the same order as the data
    print('Starting process training data')
    generator = datagen.flow(
        x_train,
        batch_size=batch_size,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size, verbose = 1)
    print(bottleneck_features_train.dtype)
    np.save(open('../models/bottleneck_features_train.npy', 'wb'),
            bottleneck_features_train)
    np.save(open('../models/bottleneck_labels_train.npy', 'wb'),
            y_train)
    
    print('Starting process validation data')
    generator = datagen.flow(
        x_valid,
        batch_size=batch_size,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size, verbose = 1)
    np.save(open('../models/bottleneck_features_validation.npy', 'wb'),
            bottleneck_features_validation)
    np.save(open('../models/bottleneck_labels_validation.npy', 'wb'),
            y_valid)

save_bottlebeck_features()

## Train top model and save parameters

In [12]:
top_model_weights_path = '../models/vgg16_bottleneck_fc_model_sgd_temp.h5'
train_data = np.load(open('../models/bottleneck_features_train.npy', 'rb'))
train_labels = np.load(open('../models/bottleneck_labels_train.npy', 'rb'))
train_labels = train_labels[:len(train_data)]
print(train_data.shape)
validation_data = np.load(open('../models/bottleneck_features_validation.npy', 'rb'))
validation_labels = np.load(open('../models/bottleneck_labels_validation.npy', 'rb'))
validation_labels = validation_labels[:len(validation_data)]

model = Sequential()
print(train_data.shape[1:])
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(17, activation='sigmoid'))
    
sgd_1 = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
sgd_2 = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
sgd_3 = SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)
    #model.compile(optimizer='adam',
     #             loss='binary_crossentropy', metrics=['accuracy'])
model.compile(optimizer=sgd_1, loss='binary_crossentropy', metrics=['accuracy'])

es = EarlyStopping(monitor='val_loss', min_delta=0.0002, patience=2, verbose=1, mode='auto')

print(train_data.shape)
print(train_labels.shape)
print(validation_data.shape)
print(validation_labels.shape)
model.fit(train_data, train_labels,
              epochs=300,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels),
             callbacks=[es])
model.save_weights(top_model_weights_path)

(34304, 4, 4, 512)
(4, 4, 512)
(34304, 4, 4, 512)
(34304, 17)
(5888, 4, 4, 512)
(5888, 17)
Train on 34304 samples, validate on 5888 samples
Epoch 1/300
34304/34304 [==============================] - 8s - loss: 0.2557 - acc: 0.9026 - val_loss: 0.2137 - val_acc: 0.9160
Epoch 2/300
34304/34304 [==============================] - 8s - loss: 0.2184 - acc: 0.9153 - val_loss: 0.1985 - val_acc: 0.9235
Epoch 3/300
34304/34304 [==============================] - 8s - loss: 0.2048 - acc: 0.9203 - val_loss: 0.1892 - val_acc: 0.9274
Epoch 4/300
34304/34304 [==============================] - 8s - loss: 0.1955 - acc: 0.9235 - val_loss: 0.1825 - val_acc: 0.9296
Epoch 5/300
34304/34304 [==============================] - 8s - loss: 0.1887 - acc: 0.9259 - val_loss: 0.1776 - val_acc: 0.9314
Epoch 6/300
34304/34304 [==============================] - 8s - loss: 0.1839 - acc: 0.9276 - val_loss: 0.1736 - val_acc: 0.9333
Epoch 7/300
34304/34304 [==============================] - 8s - loss: 0.1795 - acc: 0.9294 -

Layer Tweek

In [15]:
es = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=1, mode='auto')
model.compile(optimizer=sgd_2, loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_data, train_labels,
              epochs=300,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels),
            callbacks=[es])
model.save_weights(top_model_weights_path)

Train on 34304 samples, validate on 5888 samples
Epoch 1/300
34304/34304 [==============================] - 8s - loss: 0.1493 - acc: 0.9414 - val_loss: 0.1459 - val_acc: 0.9435
Epoch 2/300
34304/34304 [==============================] - 8s - loss: 0.1492 - acc: 0.9416 - val_loss: 0.1459 - val_acc: 0.9433
Epoch 3/300
34304/34304 [==============================] - 8s - loss: 0.1494 - acc: 0.9416 - val_loss: 0.1460 - val_acc: 0.9435
Epoch 4/300
34304/34304 [==============================] - 8s - loss: 0.1494 - acc: 0.9415 - val_loss: 0.1457 - val_acc: 0.9435
Epoch 5/300
34304/34304 [==============================] - 8s - loss: 0.1490 - acc: 0.9416 - val_loss: 0.1456 - val_acc: 0.9433
Epoch 6/300
34304/34304 [==============================] - 8s - loss: 0.1490 - acc: 0.9416 - val_loss: 0.1455 - val_acc: 0.9436
Epoch 7/300
34304/34304 [==============================] - 8s - loss: 0.1490 - acc: 0.9418 - val_loss: 0.1453 - val_acc: 0.9436
Epoch 8/300
34304/34304 [==============================

In [14]:
es = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=1, mode='auto')
model.compile(optimizer=sgd_3, loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_data, train_labels,
              epochs=300,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels),
            callbacks=[es])
model.save_weights(top_model_weights_path)

Train on 34304 samples, validate on 5888 samples
Epoch 1/300
34304/34304 [==============================] - 8s - loss: 0.1494 - acc: 0.9413 - val_loss: 0.1461 - val_acc: 0.9430
Epoch 2/300
34304/34304 [==============================] - 8s - loss: 0.1496 - acc: 0.9415 - val_loss: 0.1461 - val_acc: 0.9430
Epoch 3/300
34304/34304 [==============================] - 8s - loss: 0.1496 - acc: 0.9414 - val_loss: 0.1460 - val_acc: 0.9431
Epoch 4/300
34304/34304 [==============================] - 8s - loss: 0.1497 - acc: 0.9414 - val_loss: 0.1461 - val_acc: 0.9431
Epoch 00003: early stopping


# Difine VGG model
- Load top layer parameters
- Load VGG parameters
- Define optimiser


In [7]:
# build the VGG16 network
top_model_weights_path = '../models/vgg16_bottleneck_fc_model_sgd.h5'
batch_size = 128
input_tensor = Input(shape=(150,150,3))
base_model = applications.VGG16(weights='imagenet',include_top= False,input_tensor=input_tensor)
#last = base_model.output
print('Model loaded.')
#print(last)

# build a classifier model to put on top of the convolutional model
#x = Flatten(input_shape=(4,4,512))(last)

top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(4096, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(4096, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(17, activation='sigmoid'))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights(top_model_weights_path)

# add the model on top of the convolutional base
#x = Flatten()(last)
#x = Dense(1024, activation='relu')(x)
#preds = Dense(200, activation='softmax')(x)

#model = Model(initial_model.input, preds)
model = Model(input= base_model.input, output= top_model(base_model.output))
#model.add(top_model)

# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
#for layer in model.layers[:25]:
#    layer.trainable = False
    
    
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=SGD(lr=1e-3, momentum=0.9),
              metrics=['accuracy'])


# Data aug from simple attempt

Model loaded.


/usr/lib/python3.4/dist-packages/ipykernel_launcher.py:32: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("se...)`


In [10]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

train_generator = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest')

train_genorator_loaded = []

file_path = '../models/vgg_cnn_weights_{epoch:02d}_{val_loss:.2f}.hdf5'
model_checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=0, save_best_only=False, 
                                save_weights_only=False, mode='auto', period=1)

es = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=1, mode='auto')

datagen = ImageDataGenerator(rescale=1. / 255)
valid_generator = datagen.flow(
        x_valid, y_valid,
        batch_size=batch_size,
        shuffle=False)

In [ ]:
model.fit_generator(train_generator.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=len(x_train) / batch_size, epochs=100, validation_data=valid_generator,
                    validation_steps = len(x_valid) / batch_size,
                    callbacks = [model_checkpoint, es], verbose=1)

Epoch 1/100
229/228 [==============================] - 913s - loss: 0.1457 - acc: 0.9434 - val_loss: 0.1261 - val_acc: 0.9518
Epoch 2/100
229/228 [==============================] - 893s - loss: 0.1317 - acc: 0.9496 - val_loss: 0.1221 - val_acc: 0.9540
Epoch 3/100
166/228 [====================>.........] - ETA: 230s - loss: 0.1271 - acc: 0.9514

In [14]:
model.compile(loss='binary_crossentropy',
              optimizer=SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

model.fit_generator(train_generator.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=len(x_train) / batch_size, epochs=100, validation_data=valid_generator,
                    validation_steps = len(x_valid) / batch_size,
                    callbacks = [model_checkpoint, es], verbose=1, initial_epoch = 20)



Epoch 21/100
229/228 [==============================] - 899s - loss: 0.1038 - acc: 0.9603 - val_loss: 0.0989 - val_acc: 0.9620
Epoch 22/100
229/228 [==============================] - 899s - loss: 0.1031 - acc: 0.9609 - val_loss: 0.0988 - val_acc: 0.9622
Epoch 23/100
229/228 [==============================] - 899s - loss: 0.1033 - acc: 0.9609 - val_loss: 0.0987 - val_acc: 0.9621
Epoch 24/100
229/228 [==============================] - 899s - loss: 0.1032 - acc: 0.9608 - val_loss: 0.0988 - val_acc: 0.9620
Epoch 25/100
229/228 [==============================] - 899s - loss: 0.1036 - acc: 0.9607 - val_loss: 0.0990 - val_acc: 0.9621
Epoch 26/100
229/228 [==============================] - 899s - loss: 0.1028 - acc: 0.9610 - val_loss: 0.0989 - val_acc: 0.9619
Epoch 00025: early stopping


In [30]:

file_path = '../models/vgg_cnn_weights_{epoch:02d}_{val_loss:.2f}.hdf5'
model_checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=0, save_best_only=False, 
                                save_weights_only=False, mode='auto', period=1)

model.compile(loss='binary_crossentropy',
              optimizer=SGD(lr=1e-1, momentum=0.9),
              metrics=['accuracy'])

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit_generator(train_generator.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=0.5 * len(x_train) / batch_size, epochs=10, validation_data=valid_generator,
                    validation_steps = len(x_valid) / batch_size,
                    callbacks = [model_checkpoint, es], verbose=1)

Epoch 1/10
135/134 [==============================] - 263s - loss: 0.1575 - acc: 0.9386 - val_loss: 0.1457 - val_acc: 0.9435
Epoch 2/10
 16/134 [==>...........................] - ETA: 135s - loss: 0.1549 - acc: 0.9402

KeyboardInterrupt: 

In [15]:
from sklearn.metrics import fbeta_score

p_valid = model.predict(x_valid/255., batch_size=128)
print(fbeta_score(y_valid, np.array(p_valid) > 0.2, beta=2, average='samples'))

0.91713348386
